In [1]:
#-----target

#Personal Loan  ( 0 or 1 의 값을 갖는 변수이다. )

#-----feature

#Experience 경력
#Income 수입
#Famliy 가족단위
#CCAvg 월 카드사용량
#Education 교육수준 (1: undergrad; 2, Graduate; 3; Advance )
#Mortgage 가계대출
#Securities account 유가증권계좌유무
#CD account 양도예금증서 계좌 유무
#Online 온라인계좌유무
#CreidtCard 신용카드유무

In [4]:
# 필요한 Library
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
import statsmodels.api as sm
import matplotlib.pyplot as plt
import time

# Personal Loan 데이터 불러오기

ploan = pd.read_csv("/content/Personal_Loan.csv")

In [5]:
ploan

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2496,46,22,70,94701,4,1.9,1,212,0,0,0,0,1
2496,2497,63,37,32,93117,3,0.7,2,141,0,0,0,0,0
2497,2498,33,9,14,95133,3,0.9,3,114,0,0,0,0,0
2498,2499,38,14,111,92028,2,6.1,1,326,0,0,0,0,0


상수항의 중요성
* 절편(b0) 역할: 상수항 b0은 모델의 절편을 나타냅니다. 이는 예측 변수들의 값이 모두 0일 때의 로그 오즈(log-odds)를 나타냅니다. 즉, 다른 변수들의 영향을 받지 않았을 때의 기본적인 사건 발생 확률을 반영합니다.
* 데이터 오프셋 조정: 상수항을 추가함으로써 모델은 데이터의 기본 수준이나 오프셋(offset)을 조정할 수 있습니다. 이를 통해 모델은 변수들의 관계를 더 정확하게 학습할 수 있습니다.

In [6]:
# 사용하지 않을 변수 제거 ID, zip code
ploan_processed = ploan.dropna().drop(["ID","ZIP Code"], axis=1, inplace = False)

# 상수항 추가 (회귀모델에서 b0를 위한 상수항)
ploan_processed = sm.add_constant(ploan_processed, has_constant="add")
# sm.add_constant: statsmodels 라이브러리의 함수로, 데이터셋에 상수항을 추가합니다.
# has_constant="add": 데이터셋에 상수항이 이미 존재하지 않는 경우에만 추가하라는 의미입니다.


# 데이터 확인
ploan_processed.head()

,const,Age,Experience,Income,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1.0,25,1,49,4,1.6,1,0,0,1,0,0,0
1,1.0,45,19,34,3,1.5,1,0,0,1,0,0,0
2,1.0,39,15,11,1,1.0,1,0,0,0,0,0,0
3,1.0,35,9,100,1,2.7,2,0,0,0,0,0,0
4,1.0,35,8,45,4,1.0,2,0,0,0,0,0,1


In [7]:
# 대출여부는 1 or 0으로 결정된다. 즉 target이 1/0의 값을 갖기에
# 로지스틱 회귀모형이 적합하다.

# 여기서 사용된 ploan_processed.columns.difference(["Personal Loan"])는 pandas 라이브러리의 DataFrame에서
# 특정 컬럼을 제외한 나머지 컬럼들을 선택하는 데 사용됩니다.
feature_columns = ploan_processed.columns.difference(["Personal Loan"])
X = ploan_processed[feature_columns]
y = ploan_processed["Personal Loan"]
train_x, test_x, train_y, test_y = train_test_split(X, y, stratify=y, train_size=0.7, test_size=0.3, random_state=1)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(1750, 12) (750, 12) (1750,) (750,)


In [8]:
# 로지스틱 모형 적합

model = sm.Logit(train_y, train_x)
results = model.fit()    # 다변수함수에 뉴턴방법을 적용한 로지스틱 회귀모형

Optimization terminated successfully.
         Current function value: 0.135959
         Iterations 9


In [15]:
results.summary()

# 1. 확인내용 - p>|z|가 0.05이상이면 변수에서 제외하고 다시 모델 적합

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          Personal Loan   No. Observations:                 1750
Model:                          Logit   Df Residuals:                     1738
Method:                           MLE   Df Model:                           11
Date:                Tue, 23 Jan 2024   Pseudo R-squ.:                  0.5881
Time:                        06:35:56   Log-Likelihood:                -237.93
converged:                       True   LL-Null:                       -577.63
Covariance Type:            nonrobust   LLR p-value:                1.397e-138
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Age                   -0.0016      0.101     -0.016      0.987      -0.200       0.196
CCAvg                  0.0954      0.063      1.526      0.127      -0.027       0.218
CD Account             4.3674      0.531      8.226      0.000       3.327       5.408
CreditCard            -1.3552      0.349     -3.881      0.000      -2.040      -0.671
Education              1.4684      0.180      8.162      0.000       1.116       1.821
Experience             0.0165      0.101      0.163      0.870      -0.181       0.214
Family                 0.7160      0.125      5.734      0.000       0.471       0.961
Income                 0.0499      0.004     12.500      0.000       0.042       0.058
Mortgage               0.0006      0.001      0.634      0.526      -0.001       0.002
Online                -0.4918      0.262     -1.880      0.060      -1.005       0.021
Securities Account    -1.4553      0.484     -3.007      0.003      -2.404      -0.507
const                -12.5215      2.730     -4.586      0.000     -17.873      -7.170
======================================================================================
"""

In [ ]:
# Age와 CCAVG와 Experience가 유의하지 않음

In [9]:
# 오즈비
results.params # 회기계수

Age                   -0.001631
CCAvg                  0.095392
CD Account             4.367360
CreditCard            -1.355179
Education              1.468420
Experience             0.016455
Family                 0.715961
Income                 0.049919
Mortgage               0.000596
Online                -0.491809
Securities Account    -1.455320
const                -12.521527
dtype: float64

In [10]:
# 오즈비 계산 및 해석
np.exp(results.params)

Age                    0.998370
CCAvg                  1.100090
CD Account            78.835214
CreditCard             0.257901
Education              4.342370
Experience             1.016592
Family                 2.046153
Income                 1.051186
Mortgage               1.000596
Online                 0.611519
Securities Account     0.233326
const                  0.000004
dtype: float64

* Age (나이): 오즈비는 약 0.998로, 나이가 한 살 증가할 때마다 대출 승인 확률이 약간 감소합니다.
* CCAvg (월별 신용카드 사용량): 오즈비는 약 1.1로, 월별 신용카드 사용량이 증가하면 대출 승인 확률이 증가합니다.
* CD Account (CD 계좌 보유 여부): 오즈비는 약 78.84로, CD 계좌를 보유하고 있으면 대출 승인 확률이 크게 증가합니다.
* CreditCard (신용카드 보유 여부): 오즈비는 약 0.258로, 신용카드를 보유하고 있지 않은 경우 대출 승인 확률이 증가합니다.
* Education (교육 수준): 오즈비는 약 4.34로, 교육 수준이 높을수록 대출 승인 확률이 증가합니다.
* Experience (경험): 오즈비는 약 1.017로, 경험이 많을수록 대출 승인 확률이 약간 증가합니다.
* Family (가족 구성원 수): 오즈비는 약 2.05로, 가족 구성원이 많을수록 대출 승인 확률이 증가합니다.
* Income (소득): 오즈비는 약 1.05로, 소득이 높을수록 대출 승인 확률이 증가합니다.
* Mortgage (주택 담보 대출 금액): 오즈비는 약 1.0006로, 주택 담보 대출 금액이 증가하면 대출 승인 확률이 약간 증가합니다.
* Online (온라인 뱅킹 사용 여부): 오즈비는 약 0.612로, 온라인 뱅킹을 사용하지 않는 경우 대출 승인 확률이 감소합니다.
* Securities Account (증권 계좌 보유 여부): 오즈비는 약 0.233으로, 증권 계좌를 보유하고 있지 않은 경우 대출 승인 확률이 증가합니다.

In [11]:
pred_y = results.predict(test_x)
pred_y

1816    0.005733
1195    0.102950
2085    0.005262
406     0.025856
303     0.926229
          ...   
1676    0.184499
680     0.000842
233     0.005965
2373    0.829847
2088    0.003645
Length: 750, dtype: float64

* cut_off 함수는 두 개의 매개변수를 받습니다: y는 모델에 의해 예측된 확률 값의 배열, threshold는 이진 분류를 결정하는 임계값입니다.
* 함수 내에서 y.copy()를 통해 입력된 확률 값의 배열을 복사하여 원본 데이터에 영향을 주지 않도록 합니다.
* 임계값보다 큰 확률 값을 가진 요소들은 1로 설정하고, 그렇지 않은 값들은 0으로 설정합니다.
* 최종적으로 0과 1로 구성된 이진 결과를 반환합니다.

In [12]:
# 임계값 함수

def cut_off(y, threshold) :
    Y = y.copy()  # 대문자 Y를 새로운 변수로 하여 기존의 y값에 영향이 가지 않도록 한다.
    Y[Y>threshold] = 1
    Y[Y<threshold] = 0
    return Y.astype(int)

# 여기서 pred_y는 로지스틱 회귀 모델을 통해 예측된 확률 값입니다.
# cut_off 함수를 사용하여 임계값 0.5를 적용합니다. 이는 일반적인 이진 분류에서 사용되는 기본 임계값입니다.
# 결과적으로 pred_Y는 각 관측치가 클래스 1(예: 대출 승인)에 속할 것으로 예측되면 1, 그렇지 않으면 0을 가지게 됩니다.
pred_Y = cut_off(pred_y, 0.5)
pred_Y

1816    0
1195    0
2085    0
406     0
303     1
       ..
1676    0
680     0
233     0
2373    1
2088    0
Length: 750, dtype: int64

In [13]:
# confusion matrix (분류 표를 나타낸 행렬)
# 0을 0으로 예측할 확률과 1을 1로 예측할 확률

cfmat = confusion_matrix(test_y,pred_Y)
print(cfmat)

[[667   6]
 [ 29  48]]


In [14]:
# confusion matrix accuracy(정확도) 계산함수

def acc(cfmat):
  return (cfmat[0,0] + cfmat[1,1])/(cfmat[0,0] + cfmat[1,1] + cfmat[0,1] + cfmat[1,0])

acc(cfmat)

0.9533333333333334

In [16]:
# 임계값 0~1에 따른 정확도 확인
# 임계값에 따라서 정확도가 달라질수 있기 때문에 다음과 같은 함수를 잘 사용해야함

#threshold = np.arange(0, 1, 0.1): 0부터 1 사이에서 0.1 간격으로 임계값을 생성합니다.
threshold = np.arange(0,1,0.1)
#table = pd.DataFrame(columns=['ACC']): 정확도를 저장할 DataFrame을 생성합니다.
#여기서 'ACC'는 정확도를 나타내는 열입니다.
table = pd.DataFrame(columns = ['ACC'])

for i in threshold :
  #cut_off 함수를 사용하여 임계값 i를 적용한 예측 결과를 얻습니다.
	pred_Y = cut_off(pred_y,i)
	#혼동 행렬(confusion matrix)를 계산합니다.
	cfmat = confusion_matrix(test_y,pred_Y)
	#정의된 acc 함수를 사용하여 혼동 행렬을 기반으로 정확도를 계산하고, 이를 테이블에 저장합니다.
	table.loc[i] = acc(cfmat)

table.index.name = "threshold"
table.columns.name = "performance"
table

performance,ACC
threshold,
0.0,0.102667
0.1,0.900000
0.2,0.937333
0.3,0.946667
0.4,0.953333
0.5,0.953333
0.6,0.950667
0.7,0.949333
0.8,0.942667
